In [1]:
# Balazs Balogh - 2019-03-01

import pandas as pd
import sqlite3

stops_orig = pd.read_csv('https://raw.githubusercontent.com/DatasRev/workshop-prep/master/08_Python_and_SQL/stops.txt')
stops_orig.head()

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,parent_station,wheelchair_boarding,stop_direction
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,NaN,NaN,NaN,2.0,NaN
1,002136,Fogas üzem (kapu),47.509119,19.014403,NaN,NaN,NaN,NaN,NaN
2,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,NaN,NaN,NaN,2.0,NaN
3,003002,Puskás Ferenc Stadion M,47.500368,19.103406,NaN,NaN,NaN,2.0,NaN
4,003105,Bécsi út / Vörösvári út,47.548817,19.029722,NaN,NaN,NaN,2.0,NaN


Miután betöltöttük a szükséges könyvtárakat, és a sample filet, megvizsgáljuk, hogy a nekünk kellő mezők töltve vannak-e. Az első négy mezőben nincs NaN érték, így nem szükséges adatot tisztítani.

In [2]:
stops_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5871 entries, 0 to 5870
Data columns (total 9 columns):
stop_id                5871 non-null object
stop_name              5871 non-null object
stop_lat               5871 non-null float64
stop_lon               5871 non-null float64
stop_code              600 non-null object
location_type          312 non-null float64
parent_station         462 non-null object
wheelchair_boarding    5688 non-null float64
stop_direction         4986 non-null float64
dtypes: float64(5), object(4)
memory usage: 412.9+ KB


Az első négy oszlopot külön változóba mentjük.

In [3]:
stops = stops_orig[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700
1,002136,Fogas üzem (kapu),47.509119,19.014403
2,002138,Kőbánya alsó vasútállomás,47.483139,19.127891
3,003002,Puskás Ferenc Stadion M,47.500368,19.103406
4,003105,Bécsi út / Vörösvári út,47.548817,19.029722


Hogy jól be tudjuk lőni, hogy mekkora mezőméretek kellenek, megnézzük a stop_name leghosszabb sorát.
Az SQLite épp nem kezeli a VARCHAR mögötti számértéket (pl. VARCHAR(255)), mivel TEXT-ként kezeli, de más adatbázisnál jól jöhet.
Akár egy for loopot is lehet rá írni, hogy mondja meg az összes object típusú oszlopról, hogy melyik a leghosszabb érték.

In [4]:
stops.stop_name.map(len).max()  #megnézi, hogy melyik a leghosszabb string az oszlopban

40

Defaultban a notebook könyvtárába rakja az adatbázist is. Egy if-else kell, hogy ha többször futtatjuk, akkor továbbmehessen a notebook futtatás. A notebook ezen része példa arra, hogy hogy insertálunk adatbázisba adatot. A ''' ''' páros nem csak a sortörés lehetősége miatt kell, hanem a közbenső stringek idézőjelben maradása miatt is. Példa az if-ben lévő select 'bkk_stops' része.

In [5]:
db = sqlite3.connect('bkk_gtfs.db')
cursor = db.cursor()

# ez az elso lepes, drop-oljuk a táblát, ha már van, 

if cursor.execute('''select count() from sqlite_master where name = 'bkk_stops' ''').fetchone() == (0,):

    cursor.execute(
    '''
    CREATE TABLE bkk_stops(
           stop_id TEXT PRIMARY KEY,
           stop_name TEXT,
           stop_lat REAL,
           stop_lon REAL)
    '''
    )

    db.commit()

    # aztan beinsertalunk, .values.tolist()-el kell listbe rakni a dataframe-et.
    cursor.executemany('''INSERT INTO bkk_stops(stop_id, stop_name, stop_lat, stop_lon) VALUES(?,?,?,?)''',
                      stops.values.tolist())
    db.commit()

else:
    cursor.execute('DELETE FROM bkk_stops;')
    db.commit()
    
    cursor.executemany('''INSERT INTO bkk_stops(stop_id, stop_name, stop_lat, stop_lon) VALUES(?,?,?,?)''',
                      stops.values.tolist())
    db.commit()

Teszteljük egy lekérdezéssel, a fetchall() parancs adja vissza az összes sort, de kapott egy tizes limitet.

In [6]:
cursor.execute('SELECT * FROM bkk_stops LIMIT 10').fetchall()

[('002133', 'Örs vezér tere M+H, déli tárolótér', 47.500366, 19.1357),
 ('002136', 'Fogas üzem (kapu)', 47.509119, 19.014403),
 ('002138', 'Kőbánya alsó vasútállomás', 47.483139, 19.127891),
 ('003002', 'Puskás Ferenc Stadion M', 47.500368, 19.103406),
 ('003105', 'Bécsi út / Vörösvári út', 47.548817, 19.029722),
 ('003106', 'Örs vezér tere M+H, északi tároló', 47.50516, 19.136637),
 ('003118', 'Óbuda, Bogdáni út', 47.551471, 19.041971),
 ('003691', 'Békásmegyer, fordító F2/1', 47.599622, 19.055682),
 ('004716', 'ÉD metró járműtelep,porta', 47.469651, 19.129089999999998),
 ('004948', 'Metró ÉD járműtelep (kapu)', 47.465239000000004, 19.142612)]

Beolvassuk egy DataFrame-be a teljes bkk_stops táblát. A db változót kell odaadni neki, abban van a kapcsolat az SQLite adatbázissal.

In [7]:
bkk_stops = pd.read_sql_query('SELECT * FROM bkk_stops', db)

bkk_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5871 entries, 0 to 5870
Data columns (total 4 columns):
stop_id      5871 non-null object
stop_name    5871 non-null object
stop_lat     5871 non-null float64
stop_lon     5871 non-null float64
dtypes: float64(2), object(2)
memory usage: 183.5+ KB


In [8]:
bkk_stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700
1,002136,Fogas üzem (kapu),47.509119,19.014403
2,002138,Kőbánya alsó vasútállomás,47.483139,19.127891
3,003002,Puskás Ferenc Stadion M,47.500368,19.103406
4,003105,Bécsi út / Vörösvári út,47.548817,19.029722


Listába kell rakni a dataframe-et, hogy olvasni tudja a folium

In [9]:
stops_latlon = bkk_stops[['stop_lat', 'stop_lon']].values.tolist()
stops_latlon[:10]

[[47.500366, 19.1357],
 [47.509119, 19.014403],
 [47.483139, 19.127891],
 [47.500368, 19.103406],
 [47.548817, 19.029722],
 [47.50516, 19.136637],
 [47.551471, 19.041971],
 [47.599622, 19.055682],
 [47.469651, 19.129089999999998],
 [47.465239000000004, 19.142612]]

! A folium nem része az Anacondának, a telepítése az anaconda command promptba beírt "conda install -c conda-forge folium" parancsot kell futttatni. !

Folium importálása, és egy üres map létrehozása egy budapesti koordinátával a középpontban. A popup attribútuma a Markernek teszi rá a megálló nevét a jelzésre.
Ezután jön az 5000 megálló kirajzolása, ami a méreténél fogva igényli a MarkerCluster-t, hogy összesítsen megállókat.
Opcionálisan html-be mentjük, majd ezt hívjuk meg.
A térképet is az anaconda mappájába fogja rakni.

In [10]:
import folium
from folium.plugins import MarkerCluster

folium_map = folium.Map(location=[47.500368, 19.103406],
                        zoom_start=13,
                        tiles="cartodbpositron")

# Üres map kirajzoltatása
# folium_map

HTML-be ki tudjuk menteni, működő vizualizációval, ha a folium_map.save-et kihagyjuk, illetve az utolsó cellát, a megjelenítőt.

In [11]:
marker_cluster = MarkerCluster().add_to(folium_map)

for point in range(0, len(stops_latlon[:200])):
    folium.Marker(stops_latlon[point], popup=stops['stop_name'][point]).add_to(marker_cluster)

folium_map.save("base_map.html")

folium_map

In [12]:
from IPython.display import IFrame
IFrame("base_map.html", width=700, height=450)